In [3]:
  ! pip install wandb

  Using cached https://files.pythonhosted.org/packages/98/5f/45439b4767334b868e1c8c35b1b0ba3747d8c21be77b79f09eed7aa3c72b/wandb-0.10.30-py2.py3-none-any.whl
     |████████████████████████████████| 133kB 4.0MB/s 
  Using cached https://files.pythonhosted.org/packages/32/c8/564be4d12629b912ea431f1a50eb8b3b9d00f1a0b1ceff17f266be190007/subprocess32-3.5.4.tar.gz
     |████████████████████████████████| 174kB 19.5MB/s 
  Using cached https://files.pythonhosted.org/packages/25/a6/2ecc1daa6a304e7f1b216f0896b26156b78e7c38e1211e9b798b4716c53d/shortuuid-1.0.1-py3-none-any.whl
     |████████████████████████████████| 71kB 6.8MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=7eb3e5638486f0f447f5a691b851582166c5ff22e57c43a67504614264de8e16
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=905dff1224e5fd3

In [4]:
import wandb

In [5]:
import pandas as pd
import numpy as np
import math
import random
from keras import Input, regularizers, Model
from keras.layers import Input
from keras.layers import LSTM, SimpleRNN, GRU
from keras.layers import Dense, Embedding
from keras.utils.vis_utils import plot_model

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
data = pd.read_csv ("/content/drive/MyDrive/DLassignment3/hi.translit.sampled.train.tsv", sep = '\t')

In [8]:
data.rename(columns = {'अं' : 'Hindi', 'an' : 'English'}, inplace = True)
data

,Hindi,English,3
0,अंकगणित,ankganit,3
1,अंकल,uncle,4
2,अंकुर,ankur,4
3,अंकुरण,ankuran,3
4,अंकुरित,ankurit,3
...,...,...,...
44198,ह्वेनसांग,hiuentsang,1
44199,ह्वेनसांग,hsuantsang,1
44200,ह्वेनसांग,hyensang,1
44201,ह्वेनसांग,xuanzang,1


In [9]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
for j in range(len(data)):
  try:
    input_text=data.iloc[j,1]
    target_text=data.iloc[j,0]
    target_text = "\t" + target_text + "\n"
    if(len(input_text)>=1 and len(target_text)>=2):
      input_texts.append(input_text)
      target_texts.append(target_text)
    for char in input_text:
      if char not in input_characters:
        input_characters.add(char)
    for char in target_text:
      if char not in target_characters:
        target_characters.add(char)
  except: print("error")
input_characters.add(' ')
target_characters.add(' ')

error
error


In [10]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 44201
Number of unique input tokens: 27
Number of unique output tokens: 66
Max sequence length for inputs: 20
Max sequence length for outputs: 21


In [11]:
  tmp_x_train = []
  tmp_x_train_index = random.sample([i for i in range(0,np.array(input_texts).shape[0])],math.ceil(np.array(input_texts).shape[0]*0.7))
  tmp_x_val = []
  tmp_y_train = []
  tmp_y_val = []
  for i in tmp_x_train_index:
    tmp_x_train.append(input_texts[i])
    tmp_y_train.append(target_texts[i])
  for i in range(np.array(input_texts).shape[0]):
    if i not in tmp_x_train_index:
      tmp_x_val.append(input_texts[i])
      tmp_y_val.append(target_texts[i])
  for val in tmp_x_train[0]:
    print(input_characters[np.argmax(np.array(val))],end="")
  print()
  for val in tmp_y_train[0]:
    print(target_characters[np.argmax(np.array(val))],end="")
  print()
  for val in tmp_x_val[0]:
    print(input_characters[np.argmax(val)],end="")
  print()
  for val in tmp_y_val[0]:
    print(target_characters[np.argmax(val)],end="")
  print()
  lim=int(len(tmp_x_train)*0.9)
  x_train = tmp_x_train[0:lim]
  y_train = tmp_y_train[0:lim]
  x_val = tmp_x_train[lim:]
  y_val = tmp_y_train[lim:]
  x_test = tmp_x_val
  y_test = tmp_y_val

  # x_train = np.array(sample_in[:math.ceil(np.array(sample_in).shape[0]*0.7)])
  # y_train = np.array(sample_out[:math.ceil(np.array(sample_out).shape[0]*0.7)])
  # x_val = np.array(sample_in[math.ceil(np.array(sample_in).shape[0]*0.7):])
  # y_val = np.array(sample_out[math.ceil(np.array(sample_out).shape[0]*0.7):])
  # x_train = np.array(sample_in)
  # y_train = np.array(sample_out)

In [12]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [13]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [14]:


encoder_input_data = np.zeros(
    (len(x_train), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(x_train), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(x_train), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [15]:
for i, (input_text, target_text) in enumerate(zip(x_train, y_train)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [16]:
encoder_input_val = np.zeros(
    (len(x_val), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_val = np.zeros(
    (len(x_val), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_val = np.zeros(
    (len(x_val), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [17]:
for i, (input_text, target_text) in enumerate(zip(x_val, y_val)):
    for t, char in enumerate(input_text):
        encoder_input_val[i, t, input_token_index[char]] = 1.0
    encoder_input_val[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_val[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_val[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_val[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_val[i, t:, target_token_index[" "]] = 1.0

In [18]:
encoder_input_test = np.zeros(
    (len(x_test), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_test = np.zeros(
    (len(x_test), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_test = np.zeros(
    (len(x_test), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [19]:
for i, (input_text, target_text) in enumerate(zip(x_test, y_test)):
    for t, char in enumerate(input_text):
        encoder_input_test[i, t, input_token_index[char]] = 1.0
    encoder_input_test[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_test[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_test[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_test[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_test[i, t:, target_token_index[" "]] = 1.0

In [20]:
encoder_input_test.shape

(13260, 20, 27)

In [21]:
# Configure the sweep – specify the parameters to search through, the search strategy, the optimization metric et all.
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'epochs': {
            'values':[1]
            # 'values': [5,7,9, 10,11]
            # 'values':[100,50]
        },
        'weight_decay':{
           #'values': [0, 1e-1, 1e-2, 1e-3,0.00099,0.00111]
            'values':[1e-3]
          #  'values':[0.00099]
        },
         'batchSize':{
            'values':[16,32,64]
            # 'values':[8]
            # 'values':[16]
        },
        'regular_dropout':{
            #'values':[0.2,0.3,0.4]
             'values':[0.2]
            # 'values':[0.4]
        },
        'recurrent_dropout':{
            #'values':[0.2,0.3,0.4]
             'values':[0.2]
            # 'values':[0.2]
        },
        'regular_activation':{
            'values': ['sigmoid','tanh','relu']
            #  'values':['sigmoid']
            # 'values':['relu']
        },
        'recurrent_activation':{
            'values': ['sigmoid','tanh','relu']
            # 'values':['relu']
            #  'values':['sigmoid']
        },
        'no_of_hidden_layers':{
            'values':[1,2,3]
            # 'values':[2]
            # 'values':[3]
        },
        'hidden_layers_size':{
            'values':[64,128,256]
            # 'values':[256]
            # 'values':[128]
            # 'values':[16]
        },
        'cell_type':{
            'values':['RNN','GRU','LSTM']
            # 'values':['LSTM']
            # 'values':['GRU']
        }
        # 'train_word_accuracy':{
        #     'values':[0]
        # }
        # 'input_embedding_size':{
        #     'values':[16,32,64,256]
        # } 
        
        #'activation': {
            # 'values': ['relu', 'elu', 'selu', 'softmax']
         #   'values': ['sigmoid','tanh','ReLu']
        #}
        
        
    }
}

In [22]:
  sweep_id = wandb.sweep(sweep_config, entity="dl_45_47", project="Transliteration_model")
  #d7f2d3b3afd25dff68841ada514c88e687894723

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 2cp889xc
Sweep URL: https://wandb.ai/dl_45_47/Transliteration_model/sweeps/2cp889xc


**REAL**

In [23]:
def RNN_model(config):
  # Define an input sequence and process it.
  encoder_inputs = Input(shape=(None, num_encoder_tokens))
  encoder_inputs_1 = encoder_inputs
  encoder_states_1 = []
  for i in range(config.no_of_hidden_layers-1):
    encoder_inputs_1, state_1 = (SimpleRNN(config.hidden_layers_size, return_sequences=True,return_state=True,dropout=config.regular_dropout, recurrent_dropout=config.recurrent_dropout,activation=config.regular_activation))(encoder_inputs_1)
    encoder_states_1.append(state_1)
  encoder_inputs_1, state_1 = (SimpleRNN(config.hidden_layers_size, return_state=True,dropout=config.regular_dropout, recurrent_dropout=config.recurrent_dropout,activation=config.regular_activation))(encoder_inputs_1)
  encoder_states_1.append(state_1)

  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = Input(shape=(None, num_decoder_tokens))
  decoder_inputs_1 = decoder_inputs
  for i in range(config.no_of_hidden_layers):
    decoder_inputs_1, _ = (SimpleRNN(config.hidden_layers_size, return_sequences=True,return_state=True,dropout=config.regular_dropout, recurrent_dropout=config.recurrent_dropout,activation=config.regular_activation))(decoder_inputs_1, initial_state=encoder_states_1[i])
  decoder_dense = Dense(num_decoder_tokens, activation='softmax', kernel_regularizer=regularizers.l2(config.weight_decay))
  decoder_outputs = (decoder_dense)(decoder_inputs_1)

  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

  return model

In [24]:
def LSTM_model(config):
  # Define an input sequence and process it.
  encoder_inputs = Input(shape=(None, num_encoder_tokens))
  encoder_inputs_1 = encoder_inputs
  encoder_states_1 = []
  for i in range(config.no_of_hidden_layers-1):
    encoder_inputs_1, state_h_1, state_c_1 = (LSTM(config.hidden_layers_size, return_sequences=True,return_state=True,dropout=config.regular_dropout, recurrent_dropout=config.recurrent_dropout,activation=config.regular_activation, recurrent_activation=config.recurrent_activation))(encoder_inputs_1)
    encoder_state = [state_h_1, state_c_1]
    encoder_states_1.append(encoder_state)
  encoder_inputs_1, state_h_1, state_c_1 = (LSTM(config.hidden_layers_size, return_state=True,dropout=config.regular_dropout, recurrent_dropout=config.recurrent_dropout,activation=config.regular_activation, recurrent_activation=config.recurrent_activation))(encoder_inputs_1)
  encoder_state = [state_h_1, state_c_1]
  encoder_states_1.append(encoder_state)

  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = Input(shape=(None, num_decoder_tokens))
  decoder_inputs_1 = decoder_inputs
  for i in range(config.no_of_hidden_layers):
    decoder_inputs_1, _, _ = (LSTM(config.hidden_layers_size, return_sequences=True,return_state=True,dropout=config.regular_dropout, recurrent_dropout=config.recurrent_dropout,activation=config.regular_activation, recurrent_activation=config.recurrent_activation))(decoder_inputs_1, initial_state=encoder_states_1[i])
  decoder_dense = Dense(num_decoder_tokens, activation='softmax', kernel_regularizer=regularizers.l2(config.weight_decay))
  decoder_outputs = decoder_dense(decoder_inputs_1)

  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

  return model

In [25]:
def GRU_model(config):
  # Define an input sequence and process it.
  encoder_inputs = Input(shape=(None, num_encoder_tokens))
  encoder_inputs_1 = encoder_inputs
  encoder_states_1 = []
  for i in range(config.no_of_hidden_layers-1):
    encoder_inputs_1, state_1 = (GRU(config.hidden_layers_size, return_sequences=True,return_state=True,dropout=config.regular_dropout, recurrent_dropout=config.recurrent_dropout,activation=config.regular_activation))(encoder_inputs_1)
    encoder_states_1.append(state_1)
  encoder_inputs_1, state_1 = (GRU(config.hidden_layers_size, return_state=True,dropout=config.regular_dropout, recurrent_dropout=config.recurrent_dropout,activation=config.regular_activation))(encoder_inputs_1)
  encoder_states_1.append(state_1)

  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = Input(shape=(None, num_decoder_tokens))
  decoder_inputs_1 = decoder_inputs
  for i in range(config.no_of_hidden_layers):
    decoder_inputs_1, _ = (GRU(config.hidden_layers_size, return_sequences=True,return_state=True,dropout=config.regular_dropout, recurrent_dropout=config.recurrent_dropout,activation=config.regular_activation))(decoder_inputs_1, initial_state=encoder_states_1[i])
  decoder_dense = Dense(num_decoder_tokens, activation='softmax', kernel_regularizer=regularizers.l2(config.weight_decay))
  decoder_outputs = decoder_dense(decoder_inputs_1)

   # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

  return model

In [26]:
# The sweep calls this function with each set of hyperparameters
def train():
  # Default values for hyper-parameters we're going to sweep over
  #

    config_defaults = {
        'epochs': 5,
        'weight_decay': 1e-2,
         'batchSize':16,
        'regular_dropout':0.2,
        'recurrent_dropout':0.2,
        'no_of_hidden_layers':1,
        'hidden_layers_size':256,
        'cell_type':'LSTM'
    }

    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config

    if config.cell_type == 'RNN':
      model = RNN_model(config)
    elif config.cell_type == 'LSTM':
      model = LSTM_model(config)
    elif config.cell_type == 'GRU':
      model = GRU_model(config)
    
    # plot_model(encoder_model, to_file='encoder_model.png', show_shapes=True)
    # plot_model(decoder_model, to_file='decoder_model.png', show_shapes=True)



    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit([encoder_input_data, decoder_input_data],decoder_target_data, epochs=config.epochs,validation_data=([encoder_input_val, decoder_input_val],decoder_target_val), batch_size=config.batchSize) 

    accuracy, loss, val_accuracy, val_loss = history.history['accuracy'], history.history['loss'], history.history['val_accuracy'], history.history['val_loss']

    # wandb.log({'val_loss' : val_loss, 'val_accuracy': val_accuracy})

    for i in range(config.epochs):
      wandb.log({'val_loss' : val_loss[i], 'val_accuracy': val_accuracy[i],'loss' : loss[i], 'accuracy': accuracy[i], 'epoch': i})
    
    

In [27]:
# 18 May
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: 1tx5e1xh with config:
wandb: 	batchSize: 16
wandb: 	cell_type: GRU
wandb: 	epochs: 1
wandb: 	hidden_layers_size: 64
wandb: 	no_of_hidden_layers: 3
wandb: 	recurrent_activation: tanh
wandb: 	recurrent_dropout: 0.2
wandb: 	regular_activation: tanh
wandb: 	regular_dropout: 0.2
wandb: 	weight_decay: 0.001
wandb: Currently logged in as: dl_45_47 (use `wandb login --relogin` to force relogin)


1741/1741 [==============================] - 328s 160ms/step - loss: 1.3155 - accuracy: 0.7030 - val_loss: 0.9465 - val_accuracy: 0.7490


val_loss,0.94652
val_accuracy,0.74904
loss,1.0951
accuracy,0.72663
epoch,0
_runtime,376
_timestamp,1622017406
_step,0


val_loss,▁
val_accuracy,▁
loss,▁
accuracy,▁
epoch,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Agent Starting Run: sihxvy2w with config:
wandb: 	batchSize: 64
wandb: 	cell_type: RNN
wandb: 	epochs: 1
wandb: 	hidden_layers_size: 64
wandb: 	no_of_hidden_layers: 3
wandb: 	recurrent_activation: sigmoid
wandb: 	recurrent_dropout: 0.2
wandb: 	regular_activation: relu
wandb: 	regular_dropout: 0.2
wandb: 	weight_decay: 0.001


wandb: Ctrl + C detected. Stopping sweep.
